# Cancer prediction - starting_kit
This notebook can be consulted as an HTML page following this link.


This notebook aims at defining the theoretical understanding of cancer prediction and its correlation with genes. 

In addition to the medical knowldege, the data set which will be used will also be presented and illustrated. 

Last, an initial machine learning example will be provided in order to initiate your contribution to the challenge by proving you with an basic working solution.




In 2018 there have been 18million new cases of cancer and 10 millions death. About 20% of men will have a cancer and 17% of the woman. 1 man over 8 will die of cancer and 1 woman over 11.
Living with a cancer five year after its diagnosis is called prevalence. 50 million in the world are in this case.

The rise of cancer in our societies has multiple origins : demographic growth, the ageing of the population, social and economical factors. The type of cancers shifts with the development of countries from cancers linked to poverty to cancers associated with industrialized way of life. 

Europe concentrates 23.4% of the cancers and 20.3% of the death due to cancer even though it only represents 9% of the worldwide population. United States and Canada together count for 21% of the cancer cases, 14% of the death with 13.3% of the population. 


A cancer is caused by a mutation of the DNA, thus, the cell’s reproduction becomes out of control. Many mutation can have the same effect, resulting in many different types of cancer. That is why cancer cannot can be cured as simply as other pathologies. 

One of the major aspects of treating a cancer is detecting is, knowing which mutation of the DNA it is, what are the specific consequences of those mutations on the cells which then gives the doctor an indication about how to treat it. 


**Paragraphe à propos de la récupération des datas par Joël**
Expliquer que gdc_cancer est une aggrégation de multiples db ce qui donne des données à la fois larges mais aussi clairsemées par endroit. 

**Treatment of the data**

The data obtained through the above process gave us 5 datasets :
- df_case: General informations about the patient, age, sexe, ethnicity, multiple risk factors, diagnosis informations, what treatment has been given, the stage of the tumor, caracteristics of the tumor...etc
- cf_cnv: 
- dg_gene: Top 50 of the most mutated genes
- df_ssm: Simple somatic mutation. It gives the type of mutations, the allele of reference, the genomic DNA change...etc

As the gdc cancer is an aggregation of about 15 databases, there are many valuable informations, but most of the time it is very sparse. Thus, most fields in our datasets are empty. However, this does not imply that we cannot extract value from this data. First, we shall explore, clean


In [1]:
import pandas as pd
import numpy as np 

In [2]:
# We import the data that has already been downloaded
df_case = pd.read_csv('data/case.tsv', sep='\t')
df_cnv = pd.read_csv('data/cnv.tsv', sep='\t')
df_gene = pd.read_csv('data/gene.tsv', sep='\t')
df_ssm = pd.read_csv('data/ssm.tsv', sep='\t')

E:\Programmes\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,25,28,29,51,53,54,55,56,57,60,61,62,87,97,119,139,140,148,151,154,156,157,161,162,163,165,166,168,171,175,176,177,185,186,187,189,190,192,199,200,201,209,210,211,213,214,216,219,223,224,225,233,234,235,237,238,239,241,242,243,244,245,246,247,248,249,251,258,259,260,268,269,270,272,273,275,282,283,284,292,293,294,296,297,299,302,306,307,316,317,318,320,321,323,326,330,331,340,342,344,345,347,353,354,355,364,365,366,368,369,371,374,378,379,388,389,390,392,393,410,416,419,428,429,437,440,441,444,446,448,449,450,452,455,467,470,472,473,488,491,492,503,506,508,509,510,511,514,515,519,523,524,527,528,539,542,544,545,546,547,550,551,555,559,560,563,564,575,578,580,581,582,583,586,587,591,595,596,599,600,611,614,616,618,619,622,623,627,631,632,635,636,647,654,655,658,659,663,667,668,672,683,690,691,694,695,699,703,704,707,708,719,726

### Data Frame: case

Information on each case such as age, alcohol, tobacco, BMI (Body Mass Index), most of the values are not availale (NaN)

36 columns and 9115 rows.

The case_id column will be the key for the merge with the other dataframes.

First, let's start by looking at how filled the dataset is. 

In [3]:
serie = df_case.notnull().mean()

for name,val in zip(serie.index,serie): 
    print(name,'\t',val)

annotations.0.annotation_id 	 0.003181568842567197
annotations.0.case_id 	 0.003181568842567197
annotations.0.category 	 0.003181568842567197
annotations.0.classification 	 0.003181568842567197
annotations.0.created_datetime 	 0.003181568842567197
annotations.0.entity_id 	 0.003181568842567197
annotations.0.entity_submitter_id 	 0.003181568842567197
annotations.0.entity_type 	 0.003181568842567197
annotations.0.notes 	 0.003181568842567197
annotations.0.state 	 0.003181568842567197
annotations.0.status 	 0.003181568842567197
annotations.0.submitter_id 	 0.003181568842567197
annotations.0.updated_datetime 	 0.003181568842567197
annotations.1.annotation_id 	 0.0006582556226001097
annotations.1.case_id 	 0.0006582556226001097
annotations.1.category 	 0.0006582556226001097
annotations.1.classification 	 0.0006582556226001097
annotations.1.created_datetime 	 0.0006582556226001097
annotations.1.entity_id 	 0.0006582556226001097
annotations.1.entity_submitter_id 	 0.0006582556226001097
annota

We can see many of the columns contain almost exclusively null values. 
Thus, to have a better quality information and not empty, we chose arbitrarly to delete every column for which the filling was under 10%.
Moreover, many columns containing ids/code are useless to our project. Indeed, knowing where the tissue has been taken from a patient will not help us to predict the treatment. 

In [4]:
df_case = df_case.drop(df_case.columns[df_case.notnull().mean()<0.1], axis=1)
columns_to_drop = ['tissue_source_site.code', 'tissue_source_site.project','tissue_source_site.name', 'diagnoses.0.treatments.1.state', 'diagnoses.0.state', 'diagnoses.0.treatments.0.state', 'exposures.0.state', 'samples.0.state', 'samples.1.state', 'samples.2.state', 'demographic.demographic_id', 'demographic.state', 'demographic.submitter_id', 'demographic.updated_datetime', 'diagnoses.0.diagnosis_id', 'diagnoses.0.icd_10_code', 'diagnoses.0.submitter_id', 'diagnoses.0.treatments.1.updated_datetime', 'diagnoses.0.treatments.1.submitter_id', 'diagnoses.0.treatments.0.updated_datetime', 'diagnoses.0.updated_datetime', 'exposures.0.exposure_id', 'exposures.0.submitter_id', 'exposures.0.updated_datetime', 'samples.0.created_datetime', 'samples.0.sample_id', 'samples.0.sample_type_id', 'samples.0.submitter_id', 'samples.0.updated_datetime', 'samples.1.sample_id', 'samples.1.sample_type_id', 'samples.1.submitter_id', 'samples.2.sample_id', 'samples.2.sample_type_id', 'samples.2.submitter_id', 'tissue_source_site.bcr_id', 'submitter_id', 'diagnoses.0.treatments.0.submitter_id', 'diagnoses.0.treatments.1.treatment_id', 'samples.2.updated_datetime', 'samples.1.updated_datetime', 'diagnoses.0.treatments.0.treatment_id', 'demographic.created_datetime', 'diagnoses.0.created_datetime', 'tissue_source_site.tissue_source_site_id']
df_case = df_case.drop(columns_to_drop, axis=1)
serie = df_case.notnull().mean()

for name,val in zip(serie.index,serie): 
    print(name,'\t',val)

case_id 	 1.0
demographic.age_at_index 	 0.6760285244103127
demographic.days_to_birth 	 0.6744925946242457
demographic.ethnicity 	 0.9998902907295667
demographic.gender 	 0.9998902907295667
demographic.race 	 0.9998902907295667
demographic.vital_status 	 0.9998902907295667
demographic.year_of_birth 	 0.12024136039495337
diagnoses.0.age_at_diagnosis 	 0.4365331870543061
diagnoses.0.ajcc_pathologic_m 	 0.12046077893582008
diagnoses.0.ajcc_pathologic_n 	 0.12046077893582008
diagnoses.0.ajcc_pathologic_stage 	 0.13395501919912234
diagnoses.0.ajcc_pathologic_t 	 0.12046077893582008
diagnoses.0.ajcc_staging_system_edition 	 0.1049917718047175
diagnoses.0.classification_of_tumor 	 0.4185408667032364
diagnoses.0.days_to_diagnosis 	 0.12057048820625343
diagnoses.0.days_to_last_follow_up 	 0.10905101481075151
diagnoses.0.last_known_disease_status 	 0.975973669775096
diagnoses.0.morphology 	 0.9998902907295667
diagnoses.0.primary_diagnosis 	 0.9998902907295667
diagnoses.0.prior_malignancy 	 0.133

### Data Frame: cnv
Information on the nature of the gene mutation. Most of the value are strings.

12 columns and 1000 rows.



Once againe, we observe some useless columns such as cnv_id or id. Thus, we will drop them to lighten the dataframe and make it more readable.

We end up having 10 columns and 1000 rows. 



In [5]:
columns_to_drop = ['cnv.cnv_id', 'id']
df_cnv.drop(columns_to_drop, axis=1, inplace=True)
df_cnv.notnull().mean()

case.case_id                        1.0
cnv.chromosome                      1.0
cnv.cnv_change                      1.0
cnv.consequence.0.consequence_id    1.0
cnv.consequence.0.gene.symbol       1.0
cnv.end_position                    1.0
cnv.gene_level_cn                   1.0
cnv.ncbi_build                      1.0
cnv.start_position                  1.0
cnv.variant_status                  1.0
dtype: float64

In [6]:
df_cnv.head()

,case.case_id,cnv.chromosome,cnv.cnv_change,cnv.consequence.0.consequence_id,cnv.consequence.0.gene.symbol,cnv.end_position,cnv.gene_level_cn,cnv.ncbi_build,cnv.start_position,cnv.variant_status
0,c5891c7a-fd24-4891-8f51-373967e4a6dd,17,Loss,99c68f83-9514-5628-9623-0ba6095dd90f,BCAS3,61392838,True,GRCh38,60677453,Tumor only
1,d5f2b85a-94a9-4168-bbe8-149ec71342b0,17,Gain,a7e6f054-f5a1-5e7f-a0c0-8daa7373a434,ABCA8,68955392,True,GRCh38,68867292,Tumor only
2,eafa45a4-2fd1-4fa3-860c-9d52ed382b7d,11,Gain,924c27ca-11cf-5b13-95b9-cf518abb142d,ORAOV1,69675416,True,GRCh38,69653076,Tumor only
3,dd0f8e5a-383d-4f03-bb71-0e115d4e442c,11,Gain,00461981-a326-573d-a9a1-ad82872a1aea,SHANK2,71252577,True,GRCh38,70467856,Tumor only
4,233b02f3-c4f0-4a67-9db5-e68d5cdaccb6,8,Gain,c6208422-2c76-553a-b0eb-4472e816381f,MYC,127741434,True,GRCh38,127735434,Tumor only


### Data Frame: gene

Information on top 50 genes to studied. Most of the value are strings.

2 columns and 50 rows.

The **symbol** column will be the key to merge with the df_cnv dataframe.

What is 'score' ?

In [7]:
df_gene.head(10)

,_score,symbol
0,451.0,PIK3CA
1,404.0,TP53
2,322.0,MUC4
3,320.0,SHANK2
4,318.0,ANO1
5,316.0,ORAOV1
6,314.0,CCND1
7,314.0,DNAJB6P5
8,310.0,FGF19
9,310.0,PPFIA1


In [8]:
df_gene.dtypes
df_gene.notnull().mean()

_score    1.0
symbol    1.0
dtype: float64

### Data Frame: ssm

Information on the simple somatic mutation.

74 columns and 1000 rows.

Once again, they are many columns to be deleted either because they are empty or because they are of no use.

In [9]:
for col in df_ssm.columns: 
    print(col)

case.case_id
id
ssm.chromosome
ssm.consequence.0.consequence_id
ssm.consequence.0.transcript.gene.symbol
ssm.consequence.1.consequence_id
ssm.consequence.1.transcript.gene.symbol
ssm.consequence.10.consequence_id
ssm.consequence.10.transcript.gene.symbol
ssm.consequence.11.consequence_id
ssm.consequence.11.transcript.gene.symbol
ssm.consequence.12.consequence_id
ssm.consequence.12.transcript.gene.symbol
ssm.consequence.13.consequence_id
ssm.consequence.13.transcript.gene.symbol
ssm.consequence.14.consequence_id
ssm.consequence.14.transcript.gene.symbol
ssm.consequence.15.consequence_id
ssm.consequence.15.transcript.gene.symbol
ssm.consequence.16.consequence_id
ssm.consequence.16.transcript.gene.symbol
ssm.consequence.17.consequence_id
ssm.consequence.17.transcript.gene.symbol
ssm.consequence.18.consequence_id
ssm.consequence.18.transcript.gene.symbol
ssm.consequence.19.consequence_id
ssm.consequence.19.transcript.gene.symbol
ssm.consequence.2.consequence_id
ssm.consequence.2.transcript

We are deleting the unecessary columns.

In [10]:
df_ssm.drop(df_ssm.columns[df_ssm.columns.str.contains('ce_id')], axis=1, inplace=True)
df_ssm.drop(df_ssm.columns[df_ssm.columns.str.contains('[0-2][1-9]')], axis=1, inplace=True)
df_ssm.drop(df_ssm.columns[df_ssm.columns.str.contains('[1-9]')], axis=1, inplace=True)
columns_to_drop = ["id"]
df_ssm.drop("id", axis=1,inplace=True)

print(df_ssm.shape)
df_ssm.head(5)



(2347, 13)


,case.case_id,ssm.chromosome,ssm.consequence.0.transcript.gene.symbol,ssm.cosmic_id.0,ssm.end_position,ssm.genomic_dna_change,ssm.mutation_subtype,ssm.mutation_type,ssm.ncbi_build,ssm.reference_allele,ssm.ssm_id,ssm.start_position,ssm.tumor_allele
0,d2694b09-7844-4496-865c-5d3d8cd03330,chr3,MUC4,NaN,195780168,chr3:g.195780168A>T,Single base substitution,Simple Somatic Mutation,GRCh38,A,92dded19-d0ce-5c34-9d99-169342f9bc37,195780168,T
1,17baef7c-d97d-4b98-ab53-503ef856523d,chr3,PIK3CA,COSM446001,179203764,chr3:g.179203764A>C,Single base substitution,Simple Somatic Mutation,GRCh38,A,7ea2ecba-9bbc-5d84-8a58-2ed20f13bdf3,179203764,C
2,d3d545b3-457f-4389-821f-704cb24aff7f,chr17,TP53,COSM1480087,7676042,chr17:g.7676041_7676042insGA,Small insertion,Simple Somatic Mutation,GRCh38,'-,1c4dad33-40cc-58d0-9d8c-4d808cb33401,7676041,GA
3,55262fcb-1b01-4480-b322-36570430c917,chr17,ABCA8,NaN,68883820,chr17:g.68883820C>G,Single base substitution,Simple Somatic Mutation,GRCh38,C,905a9d14-3a2e-5fb8-994e-48a2e5a1fa54,68883820,G
4,91b09291-5944-4691-a0d8-7d1ccc649b9b,chr3,MUC4,COSM3847118,195782578,chr3:g.195782578G>A,Single base substitution,Simple Somatic Mutation,GRCh38,G,eb5acb6d-ebcc-50fc-9dc0-9a7ac5b85fc4,195782578,A


It leaves us with a 13 columns dataframe which is way more exploitable.

We shall now start to merge the dataframes to get a single one that contains all the data.

In [197]:
df_ssm["ssm.mutation_subtype"].unique()

array(['Single base substitution', 'Small insertion', 'Small deletion'],
      dtype=object)

# It starts from there:

---
---

### Transcriptome (ARN)

We need 3 data tables to be able to use the transcriptomic data :

* HTseq data (in ENSEMBL, transcriptome_case_id format) $\rightarrow$ main transcriptomic data

* ENSEMBL to symbol table

* transcriptome_case_id to case_id table

#### ENSEMBL to Symbol

To get the relation table, click [this link](https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&col=md_ensembl_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit) from [https://www.genenames.org/cgi-bin/download](https://www.genenames.org/cgi-bin/download). Symbols and ENSEMBL are two conventions in naming genes and CNV, SSM data use symbols while Transcriptomic data use ENSEMBL.

Create a folder ```transcriptome``` in ```./data```, put the coresponding file in it. Rename it ```ENSEMBL.txt``` then run the following code:

In [152]:
ensembl = pd.read_csv('./data/transcriptome/ENSEMBL.txt', sep="\t", header=0)
ensembl

,Approved symbol,Ensembl ID(supplied by Ensembl)
0,A1BG,ENSG00000121410
1,A1BG-AS1,ENSG00000268895
2,A1CF,ENSG00000148584
3,A1S9T,NaN
4,A2M,ENSG00000175899
...,...,...
47479,ZYG11B,ENSG00000162378
47480,ZYX,ENSG00000159840
47481,ZYXP1,ENSG00000274572
47482,ZZEF1,ENSG00000074755


We only keep the 50 more mutated genes:

In [153]:
df_gene_exp = (pd.merge(df_gene, ensembl, 
                       how='inner', 
                       left_on='symbol', 
                       right_on='Approved symbol')
               .drop(["Approved symbol", "_score"], axis=1)
               .rename(columns={"Ensembl ID(supplied by Ensembl)": "Ensembl ID"})
              )
df_gene_exp

,symbol,Ensembl ID
0,PIK3CA,ENSG00000121879
1,TP53,ENSG00000141510
2,MUC4,ENSG00000145113
3,SHANK2,ENSG00000162105
4,ANO1,ENSG00000131620
5,CCND1,ENSG00000110092
6,DNAJB6P5,ENSG00000256441
7,DNAJB6P5,ENSG00000256441
8,FGF19,ENSG00000162344
9,PPFIA1,ENSG00000131626


We rather use a dictionnary to access rapidly the corresponding Symbol of an ENSEMBL ID:

In [125]:
match_gene = {}
for symb, ens in zip(df_gene_exp["symbol"], df_gene_exp["Ensembl ID"]):
    match_gene[ens] = symb

#### UUID matching

The UUID in the transcriptomic data corresponds to different values than the one used for the general medical data. Fortunately, we can find a correspondance in a json file downloadable from [the gdc portal website](https://portal.gdc.cancer.gov/repository?filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.analysis.workflow_type%22%2C%22value%22%3A%5B%22HTSeq%20-%20Counts%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.cases.primary_site%22%2C%22value%22%3A%5B%22breast%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.data_type%22%2C%22value%22%3A%5B%22Gene%20Expression%20Quantification%22%5D%7D%7D%5D%7D) (click on the JSON button under the pie charts).

Put the file in ```./data/transcriptome``` and name it ```UUID_match.json```.

In [111]:
import json

In [130]:
with open("./data/transcriptome/UUID_match.json", "r") as f:
    json_content = json.load(f)

match_uuid = {}
for dico in json_content:
    match_uuid[dico['file_name']] = dico['cases'][0]['case_id']


#### Download transcriptomic data

The first step consists in downloading the manifest file by clicking on the download button named **Manifest** in [this page](https://portal.gdc.cancer.gov/repository?filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.analysis.workflow_type%22%2C%22value%22%3A%5B%22HTSeq%20-%20Counts%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.cases.primary_site%22%2C%22value%22%3A%5B%22breast%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.data_type%22%2C%22value%22%3A%5B%22Gene%20Expression%20Quantification%22%5D%7D%7D%5D%7D) of the gdc portal website. The manifest contains all the information about what is wanted to be downloaded from the database (that is to say the transcriptomic data).

The second steps is to install [gdc client](https://gdc.cancer.gov/access-data/gdc-data-transfer-tool#:~:text=The%20GDC%20provides%20a%20standard%20client-based%20mechanism%20in,or%20FASTQ%2C%20make%20up%20the%20bulk%20of%20data.) to download the data. 

**Important**: set "gdc-client" to the environnement path variables by pointing to the ```gdc-client.exe``` file.

Then open a terminal and go to ```./data/transcriptome``` where is also the manifest, then run:

```
gdc-client download -m <name of manifest>.txt
```

It will created thousands of folders, and each one contains a ```.gz``` zip file containing the transcriptomic data of 1 individual.

#### Extract content

In [156]:
# add useful info of 1 individual to dict 'merge'
# it directly uses 'match_uuid' and 'match_gene' done previously
# to already store the useful data
def add_case_htseq(content, merge, filename):
    for line in content:
        line = str(line)[2:-1]
        for ens in match_gene.keys():
            if ens in line:
                htseq = line.split('\\t')[1][:-2]
                merge["case_id"].append(match_uuid[filename])  # add UUID 
                merge["gene"].append(match_gene[line.split('.')[0]])  # add gene symbol
                merge["HTseq_counts"].append(int(htseq))   # add transcriptomic data
    return merge

In [157]:
# long to run
import gzip
import os

merge_dict = {"case_id": list(), "gene": list(), "HTseq_counts": list()}
# Go through all the folders
for root, dirs, files in os.walk("./data/transcriptom", topdown=False):
    for name in files:
        if name.endswith(".gz"):   # if .gz file containing the wanted data 
            with gzip.open(os.path.join(root, name), 'r') as f:
                content = f.readlines()   # list of the lines (costly step)
                # add useful data into merge:
                merge_dict = add_case_htseq(content, merge_dict, name)   

In [159]:
df_arn = pd.DataFrame(merge_dict)
df_arn.to_csv("./data/arn.tsv")   # save the dataframe (long to run)

In [154]:
df_arn = pd.read_csv("./data/arn.tsv")
df_arn

,Unnamed: 0,case_id,gene,HTseq_counts
0,0,59bf0ce4-715a-4017-bccb-c59916e78324,TG,183
1,1,59bf0ce4-715a-4017-bccb-c59916e78324,SDK2,98
2,2,59bf0ce4-715a-4017-bccb-c59916e78324,FGF4,0
3,3,59bf0ce4-715a-4017-bccb-c59916e78324,CTTN,11975
4,4,59bf0ce4-715a-4017-bccb-c59916e78324,TRPS1,14826
...,...,...,...,...
75211,75211,76bd9bb9-9a8b-4a90-bb7c-eafe76472ee4,AMZ2,3613
75212,75212,76bd9bb9-9a8b-4a90-bb7c-eafe76472ee4,HELZ,1963
75213,75213,76bd9bb9-9a8b-4a90-bb7c-eafe76472ee4,RYR2,15
75214,75214,76bd9bb9-9a8b-4a90-bb7c-eafe76472ee4,POU5F1B,53


Now that we have all the dataframes required (df_case, df_cnv, df_ssm, df_arn), we can join them into one.

## Preparing the merge

### SSM

To avoid all the details of mutations, we need to aggregate the SSM data for any couple (case_id, gene). To do so, we decided to use a scoring of the types of mutations in terms of their expected consequences.

* **Single base substitution**: This type of mutation corresponds to the substituation of a ATGC base by an other. It can either just change the amino acid in the protein with variable changes, create or delete a stop codon that changes the size of the protein (rare), or change nothing. We attribute a score of $1$ for every such mutation.

* **Single base deletion/addition**: This type of mutation has a huge impact on the amino acid encoding since, what is called a frame-shift, can change all the amino acids after the mutation in the sequence. We attribute a score of $3$ for every such mutation.

In [14]:
def score_attribution(x):
    if x == "Single base substitution":
        return 1
    else:
        return 3

df_ssm["ssm.mutation_subtype"] = df_ssm["ssm.mutation_subtype"].apply(score_attribution)

We then sum the scores of each mutation for a given (case_id, gene):

In [16]:
df_ssm_ = (df_ssm[["case.case_id",
                  "ssm.consequence.0.transcript.gene.symbol",
                  "ssm.mutation_subtype"]]
          .groupby(by=["case.case_id", "ssm.consequence.0.transcript.gene.symbol"])
           .sum()
         )

df_ssm_.reset_index(inplace=True)  
df_ssm_

,case.case_id,ssm.consequence.0.transcript.gene.symbol,ssm.mutation_subtype
0,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,MUC4,2
1,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,PIK3CA,1
2,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,RYR2,1
3,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,TG,1
4,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,TP53,1
...,...,...,...
1837,fec0da58-1047-44d2-b6d1-c18cceed43dc,TRPS1,1
1838,fec0da58-1047-44d2-b6d1-c18cceed43dc,TTN,1
1839,ff0f875f-113a-4c0f-8c0a-4259cd496e6d,PIK3CA,1
1840,ff0f875f-113a-4c0f-8c0a-4259cd496e6d,TP53,1


We rename the columns to make things simpler:

In [17]:
df_ssm_ = df_ssm_.rename(columns={"ssm.mutation_subtype": "ssm.mutation_importance",
                       "ssm.consequence.0.transcript.gene.symbol": "ssm.symbol"})
df_ssm_

,case.case_id,ssm.symbol,ssm.mutation_importance
0,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,MUC4,2
1,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,PIK3CA,1
2,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,RYR2,1
3,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,TG,1
4,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,TP53,1
...,...,...,...
1837,fec0da58-1047-44d2-b6d1-c18cceed43dc,TRPS1,1
1838,fec0da58-1047-44d2-b6d1-c18cceed43dc,TTN,1
1839,ff0f875f-113a-4c0f-8c0a-4259cd496e6d,PIK3CA,1
1840,ff0f875f-113a-4c0f-8c0a-4259cd496e6d,TP53,1


We use pivoting to obtain a one hot encoding of every gene for a given patient:

In [18]:
df_ssm_pivot = df_ssm_.pivot(index="case.case_id",
                             columns="ssm.symbol",
                             values=["ssm.mutation_importance"])
df_ssm_pivot.reset_index(inplace=True)
df_ssm_pivot.fillna(0., inplace=True)  # We put 0 since there is no mutation
df_ssm_pivot

case.case_id ssm.mutation_importance  \
ssm.symbol                                                        ABCA10   
0           00807dae-9f4a-4fd1-aac2-82eb11bf2afb                     0.0   
1           00a2d166-78c9-4687-a195-3d6315c27574                     0.0   
2           00b11ca8-8540-4a3d-b602-ec754b00230b                     0.0   
3           011b9b2d-ebe5-42bf-9662-d922faccc7a1                     0.0   
4           01263518-5f7c-49dc-8d7e-84b0c03a6a63                     0.0   
..                                           ...                     ...   
804         fcef8cb5-fb2c-4bfb-82cd-6b9f3145182c                     0.0   
805         fe15c48b-116d-461e-9b40-68514730fd18                     0.0   
806         fe7f74b8-20f4-4471-91dc-4cca8c68e5c0                     0.0   
807         fec0da58-1047-44d2-b6d1-c18cceed43dc                     0.0   
808         ff0f875f-113a-4c0f-8c0a-4259cd496e6d                     0.0   

                                                        ...                   \
ssm.symbol ABCA6 ABCA8 ABCA9 AMZ2 ANO1 ARSG BCAS3 BPTF  ... RYR2 SDK2 SHANK2   
0            0.0   0.0   0.0  0.0  0.0  0.0   0.0  0.0  ...  1.0  0.0    0.0   
1            0.0   0.0   0.0  0.0  0.0  0.0   0.0  0.0  ...  0.0  0.0    0.0   
2            0.0   0.0   0.0  0.0  0.0  0.0   0.0  1.0  ...  0.0  0.0    0.0   
3            0.0   1.0   0.0  0.0  0.0  0.0   0.0  0.0  ...  0.0  1.0    0.0   
4            1.0   0.0   0.0  0.0  0.0  0.0   0.0  1.0  ...  0.0  0.0    0.0   
..           ...   ...   ...  ...  ...  ...   ...  ...  ...  ...  ...    ...   
804          0.0   0.0   0.0  0.0  0.0  0.0   0.0  0.0  ...  0.0  0.0    0.0   
805          0.0   0.0   0.0  0.0  0.0  0.0   0.0  0.0  ...  0.0  0.0    0.0   
806          0.0   0.0   0.0  0.0  0.0  0.0   0.0  0.0  ...  0.0  0.0    0.0   
807          0.0   0.0   3.0  0.0  0.0  0.0   0.0  0.0  ...  0.0  0.0    0.0   
808          0.0   0.0   0.0  0.0  0.0  0.0   0.0  0.0  ...  0.0  0.0    0.0   

                                                     
ssm.symbol SLC16A6 TANC2   TG TP53 TPCN2 TRPS1  TTN  
0              0.0   0.0  1.0  1.0   0.0   0.0  0.0  
1              0.0   0.0  0.0  0.0   0.0   0.0  0.0  
2              0.0   0.0  0.0  0.0   0.0   0.0  0.0  
3              0.0   0.0  0.0  0.0   0.0   0.0  0.0  
4              0.0   0.0  0.0  1.0   0.0   0.0  0.0  
..             ...   ...  ...  ...   ...   ...  ...  
804            0.0   0.0  0.0  0.0   0.0   0.0  1.0  
805            0.0   0.0  0.0  0.0   0.0   0.0  0.0  
806            0.0   0.0  0.0  0.0   0.0   0.0  0.0  
807            0.0   0.0  0.0  0.0   0.0   1.0  1.0  
808            0.0   0.0  0.0  1.0   0.0   0.0  1.0  

[809 rows x 47 columns]

In [19]:
# Flatten the multi-indexing
df_ssm_pivot.columns = df_ssm_pivot.columns.droplevel()

In [140]:
# rename every column
renames = {}
for x in df_ssm_pivot.columns:
#     print("*", x)
    if x == '':
        renames[x] = 'case.case_id'
    else:
        renames[x] = 'ssm.importance.' + x 

In [23]:
df_ssm_pivot = df_ssm_pivot.rename(columns=renames)
df_ssm_pivot

ssm.symbol,case.case_id,ssm.importance.ABCA10,ssm.importance.ABCA6,ssm.importance.ABCA8,ssm.importance.ABCA9,ssm.importance.AMZ2,ssm.importance.ANO1,ssm.importance.ARSG,ssm.importance.BCAS3,ssm.importance.BPTF,...,ssm.importance.RYR2,ssm.importance.SDK2,ssm.importance.SHANK2,ssm.importance.SLC16A6,ssm.importance.TANC2,ssm.importance.TG,ssm.importance.TP53,ssm.importance.TPCN2,ssm.importance.TRPS1,ssm.importance.TTN
0,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,00a2d166-78c9-4687-a195-3d6315c27574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00b11ca8-8540-4a3d-b602-ec754b00230b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,011b9b2d-ebe5-42bf-9662-d922faccc7a1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01263518-5f7c-49dc-8d7e-84b0c03a6a63,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,fcef8cb5-fb2c-4bfb-82cd-6b9f3145182c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
805,fe15c48b-116d-461e-9b40-68514730fd18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
806,fe7f74b8-20f4-4471-91dc-4cca8c68e5c0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
807,fec0da58-1047-44d2-b6d1-c18cceed43dc,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


### CNV

We do the the same king of aggregation with CNV except we normally have only one cnv max per gene. We still aggregate with a 'min' to manage only one if there turns out to be 2 CNVs.

In [24]:
df_cnv_ = (df_cnv[["case.case_id", "cnv.cnv_change", "cnv.consequence.0.gene.symbol"]]
           .groupby(["case.case_id", "cnv.consequence.0.gene.symbol"])
           .aggregate('min')
          )
df_cnv_.reset_index(inplace=True)
df_cnv_ = df_cnv_.rename(columns={'cnv.consequence.0.gene.symbol':
                                 'cnv.symbol'})
df_cnv_

,case.case_id,cnv.symbol,cnv.cnv_change
0,001cef41-ff86-4d3f-a140-a647ac4b10a1,ANO1,Gain
1,001cef41-ff86-4d3f-a140-a647ac4b10a1,AP001888.1,Gain
2,001cef41-ff86-4d3f-a140-a647ac4b10a1,CCND1,Gain
3,001cef41-ff86-4d3f-a140-a647ac4b10a1,CPT1A,Gain
4,001cef41-ff86-4d3f-a140-a647ac4b10a1,CTTN,Gain
...,...,...,...
12876,ff0f875f-113a-4c0f-8c0a-4259cd496e6d,ORAOV1,Gain
12877,ff0f875f-113a-4c0f-8c0a-4259cd496e6d,PPFIA1,Gain
12878,ff0f875f-113a-4c0f-8c0a-4259cd496e6d,SHANK2,Gain
12879,ff0f875f-113a-4c0f-8c0a-4259cd496e6d,TPCN2,Gain


Pivoting:

In [145]:
df_cnv_pivot = df_cnv_.pivot(index="case.case_id",
                             columns="cnv.symbol",
                             values=["cnv.cnv_change"])
df_cnv_pivot.reset_index(inplace=True)
df_cnv_pivot.fillna('None', inplace=True)
df_cnv_pivot.columns = df_cnv_pivot.columns.droplevel()

Renaming:

In [146]:
renames = {}
for x in df_cnv_pivot.columns:
#     print("*", x)
    if x == '':
        renames[x] = 'case.case_id'
    else:
        renames[x] = 'cnv.change.' + x 

In [147]:
df_cnv_pivot = df_cnv_pivot.rename(columns=renames)
df_cnv_pivot

cnv.symbol,case.case_id,cnv.change.ABCA10,cnv.change.ABCA6,cnv.change.ABCA8,cnv.change.ABCA9,cnv.change.AC108925.1,cnv.change.AMZ2,cnv.change.ANO1,cnv.change.AP001888.1,cnv.change.ARSG,...,cnv.change.RYR2,cnv.change.SDK2,cnv.change.SHANK2,cnv.change.SLC16A6,cnv.change.TANC2,cnv.change.TG,cnv.change.TP53,cnv.change.TPCN2,cnv.change.TRPS1,cnv.change.TTN
0,001cef41-ff86-4d3f-a140-a647ac4b10a1,None,None,None,None,None,None,Gain,Gain,None,...,None,None,Gain,None,None,None,None,Gain,None,None
1,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,Gain,Gain,Gain,Gain,None,Gain,None,None,Gain,...,None,Gain,None,Gain,Gain,None,None,None,None,None
2,00a2d166-78c9-4687-a195-3d6315c27574,Gain,Gain,Gain,Gain,None,Gain,None,None,Gain,...,None,Gain,None,Gain,Gain,None,None,None,None,None
3,00b11ca8-8540-4a3d-b602-ec754b00230b,None,None,None,None,None,None,Gain,Gain,None,...,None,None,None,None,None,None,None,Gain,None,None
4,011b9b2d-ebe5-42bf-9662-d922faccc7a1,None,None,None,None,None,None,None,None,None,...,Gain,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,fe15c48b-116d-461e-9b40-68514730fd18,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
794,fe2cd610-aa52-4789-ac62-7683281bb22f,None,None,None,None,None,None,Gain,Gain,None,...,None,None,Gain,None,None,None,None,Gain,None,None
795,fe7f74b8-20f4-4471-91dc-4cca8c68e5c0,None,None,None,None,None,None,None,None,None,...,None,Gain,None,None,Gain,None,None,None,None,None
796,fec0da58-1047-44d2-b6d1-c18cceed43dc,None,None,None,None,None,None,Gain,Gain,None,...,None,None,Gain,None,None,None,None,Gain,None,None


### ARN

We only need pivoting and renaming for ARN.

In [29]:
df_arn

,Unnamed: 0,case_id,gene,HTseq_counts
0,0,59bf0ce4-715a-4017-bccb-c59916e78324,TG,183
1,1,59bf0ce4-715a-4017-bccb-c59916e78324,SDK2,98
2,2,59bf0ce4-715a-4017-bccb-c59916e78324,FGF4,0
3,3,59bf0ce4-715a-4017-bccb-c59916e78324,CTTN,11975
4,4,59bf0ce4-715a-4017-bccb-c59916e78324,TRPS1,14826
...,...,...,...,...
75211,75211,76bd9bb9-9a8b-4a90-bb7c-eafe76472ee4,AMZ2,3613
75212,75212,76bd9bb9-9a8b-4a90-bb7c-eafe76472ee4,HELZ,1963
75213,75213,76bd9bb9-9a8b-4a90-bb7c-eafe76472ee4,RYR2,15
75214,75214,76bd9bb9-9a8b-4a90-bb7c-eafe76472ee4,POU5F1B,53


In [148]:
df_arn_pivot = df_arn.pivot_table(index="case_id",
                             columns="gene",
                             values="HTseq_counts",
                             aggfunc='sum'
                           )
df_arn_pivot.reset_index(inplace=True)

In [149]:
renames = {}
for x in df_arn_pivot.columns:
    if x in ['case_id']:
        renames[x] = x
    else:
        renames[x] = 'arn.HTseq_counts.' + x 

In [150]:
df_arn_pivot = df_arn_pivot.rename(columns=renames)
df_arn_pivot

gene,case_id,arn.HTseq_counts.ABCA10,arn.HTseq_counts.ABCA6,arn.HTseq_counts.ABCA8,arn.HTseq_counts.ABCA9,arn.HTseq_counts.AMZ2,arn.HTseq_counts.ANO1,arn.HTseq_counts.ARSG,arn.HTseq_counts.BCAS3,arn.HTseq_counts.BPTF,...,arn.HTseq_counts.RYR2,arn.HTseq_counts.SDK2,arn.HTseq_counts.SHANK2,arn.HTseq_counts.SLC16A6,arn.HTseq_counts.TANC2,arn.HTseq_counts.TG,arn.HTseq_counts.TP53,arn.HTseq_counts.TPCN2,arn.HTseq_counts.TRPS1,arn.HTseq_counts.TTN
0,001cef41-ff86-4d3f-a140-a647ac4b10a1,46,160,197,85,5055,4188,2270,1645,5340,...,42,368,1628,2222,4110,18,3437,1057,27924,285
1,0045349c-69d9-4306-a403-c9c1fa836644,61,127,347,194,2248,7408,823,2302,9236,...,33,3836,8336,165,1714,133,11130,758,12095,1219
2,007df8da-3d8c-497d-b7d3-3a02596f9577,111,225,217,172,2644,1289,598,1210,2906,...,30,36,1146,2870,3592,16,389,401,20801,1074
3,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,31,127,121,88,2932,1694,488,376,2349,...,83,74,1942,729,3846,18,2997,655,8320,341
4,00a2d166-78c9-4687-a195-3d6315c27574,21,143,26,68,13775,2622,1024,1966,9583,...,34,266,1274,700,9455,19,6779,1123,13940,306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368,fe2cd610-aa52-4789-ac62-7683281bb22f,57,54,20,16,4116,1046,1050,936,4704,...,2645,24,5624,1233,913,27,6768,614,25012,437
1369,fe7f74b8-20f4-4471-91dc-4cca8c68e5c0,164,367,314,299,1976,3194,462,2483,10942,...,27,405,3441,390,6497,27,4153,1563,4224,749
1370,fec0da58-1047-44d2-b6d1-c18cceed43dc,92,414,894,317,4782,5776,1640,1968,4447,...,44,207,1172,951,6078,274,3132,1021,20023,263
1371,ff0f875f-113a-4c0f-8c0a-4259cd496e6d,270,675,726,509,4611,36665,695,1016,4505,...,74,231,1224,510,3770,81,6545,1381,24171,767


### MERGE

We want to keep the individuals that have at least one info on their genetics. We use 'outer' joins first between genetic tables (SSM, CNV, ARN), then 'right' on the merge with Case, where Case is on the left.

SSM + CNV

In [155]:
df = pd.merge(df_ssm_pivot, df_cnv_pivot, how='outer', left_on='case.case_id', right_on='case.case_id')
df

,case.case_id,ssm.importance.ABCA10,ssm.importance.ABCA6,ssm.importance.ABCA8,ssm.importance.ABCA9,ssm.importance.AMZ2,ssm.importance.ANO1,ssm.importance.ARSG,ssm.importance.BCAS3,ssm.importance.BPTF,...,cnv.change.RYR2,cnv.change.SDK2,cnv.change.SHANK2,cnv.change.SLC16A6,cnv.change.TANC2,cnv.change.TG,cnv.change.TP53,cnv.change.TPCN2,cnv.change.TRPS1,cnv.change.TTN
0,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,Gain,None,Gain,Gain,None,None,None,None,None
1,00a2d166-78c9-4687-a195-3d6315c27574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,Gain,None,Gain,Gain,None,None,None,None,None
2,00b11ca8-8540-4a3d-b602-ec754b00230b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,None,None,None,None,None,None,None,Gain,None,None
3,011b9b2d-ebe5-42bf-9662-d922faccc7a1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Gain,None,None,None,None,None,None,None,None,None
4,01263518-5f7c-49dc-8d7e-84b0c03a6a63,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,None,None,Gain,None,None,None,None,Gain,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,f6eebd4b-b63a-4a9c-92d3-0d954a8a6655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,Gain,None,None,None,None,Loss,None,None
1003,f9eb88f7-9293-46bf-ace4-a746e4ff80dc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,Gain,None,None,None,None,Gain,None,None
1004,fbee40f1-d6d8-4156-8d42-36e09bb9f095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,Gain,None,None,Gain,None,Gain,Gain,None
1005,fd0b1b8b-1a76-480a-b61c-d176e4e02c6b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,Gain,None,None,Gain,None


(SSM + CNV) + ARN

In [156]:
df_arn_pivot.rename(columns={'case_id': 'case.case_id'}, inplace=True)
df = pd.merge(df, df_arn_pivot, how='outer', left_on='case.case_id', right_on='case.case_id')

df

,case.case_id,ssm.importance.ABCA10,ssm.importance.ABCA6,ssm.importance.ABCA8,ssm.importance.ABCA9,ssm.importance.AMZ2,ssm.importance.ANO1,ssm.importance.ARSG,ssm.importance.BCAS3,ssm.importance.BPTF,...,arn.HTseq_counts.RYR2,arn.HTseq_counts.SDK2,arn.HTseq_counts.SHANK2,arn.HTseq_counts.SLC16A6,arn.HTseq_counts.TANC2,arn.HTseq_counts.TG,arn.HTseq_counts.TP53,arn.HTseq_counts.TPCN2,arn.HTseq_counts.TRPS1,arn.HTseq_counts.TTN
0,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,83.0,74.0,1942.0,729.0,3846.0,18.0,2997.0,655.0,8320.0,341.0
1,00a2d166-78c9-4687-a195-3d6315c27574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.0,266.0,1274.0,700.0,9455.0,19.0,6779.0,1123.0,13940.0,306.0
2,00b11ca8-8540-4a3d-b602-ec754b00230b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,177.0,528.0,2830.0,880.0,5889.0,56.0,4543.0,3168.0,15170.0,1334.0
3,011b9b2d-ebe5-42bf-9662-d922faccc7a1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,418.0,603.0,6459.0,12415.0,17324.0,391.0,7822.0,1206.0,212884.0,19495.0
4,01263518-5f7c-49dc-8d7e-84b0c03a6a63,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,29.0,37.0,4405.0,1289.0,15729.0,20.0,5374.0,2801.0,40581.0,244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,fbad4165-ac84-44ed-8688-47f93fee5957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,125.0,564.0,571.0,2093.0,77.0,1327.0,550.0,7538.0,174.0
1375,fc774c98-c5fa-4ba8-864d-45465e55250a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,34.0,9.0,0.0,162.0,790.0
1376,fd7704f9-fd03-4b32-b0f4-98bd0444697b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4823.0,995.0,24148.0,2352.0,33958.0,1443.0,4487.0,4459.0,111836.0,102811.0
1377,fe25353b-d114-43f7-8d9b-eb6f2bd61d3d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1397.0,1193.0,11654.0,318.0,25576.0,589.0,2713.0,3141.0,100850.0,63567.0


Case + ((SSM + CNV) + ARN)

In [157]:
df.rename(columns={'case.case_id': 'case_id'}, inplace=True)
df = pd.merge(df_case, df, how='right', left_on='case_id', right_on='case_id')

df

,case_id,demographic.age_at_index,demographic.days_to_birth,demographic.ethnicity,demographic.gender,demographic.race,demographic.vital_status,demographic.year_of_birth,diagnoses.0.age_at_diagnosis,diagnoses.0.ajcc_pathologic_m,...,arn.HTseq_counts.RYR2,arn.HTseq_counts.SDK2,arn.HTseq_counts.SHANK2,arn.HTseq_counts.SLC16A6,arn.HTseq_counts.TANC2,arn.HTseq_counts.TG,arn.HTseq_counts.TP53,arn.HTseq_counts.TPCN2,arn.HTseq_counts.TRPS1,arn.HTseq_counts.TTN
0,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,50.0,-18345.0,not hispanic or latino,female,white,Alive,1955.0,18345.0,M0,...,83.0,74.0,1942.0,729.0,3846.0,18.0,2997.0,655.0,8320.0,341.0
1,00a2d166-78c9-4687-a195-3d6315c27574,56.0,-20713.0,not hispanic or latino,female,white,Alive,1954.0,20713.0,M0,...,34.0,266.0,1274.0,700.0,9455.0,19.0,6779.0,1123.0,13940.0,306.0
2,00b11ca8-8540-4a3d-b602-ec754b00230b,61.0,-22497.0,not hispanic or latino,female,white,Alive,1951.0,22497.0,MX,...,177.0,528.0,2830.0,880.0,5889.0,56.0,4543.0,3168.0,15170.0,1334.0
3,011b9b2d-ebe5-42bf-9662-d922faccc7a1,71.0,-26274.0,not hispanic or latino,female,white,Alive,1939.0,26274.0,MX,...,418.0,603.0,6459.0,12415.0,17324.0,391.0,7822.0,1206.0,212884.0,19495.0
4,01263518-5f7c-49dc-8d7e-84b0c03a6a63,76.0,-27942.0,not reported,female,not reported,Alive,1933.0,27942.0,M1,...,29.0,37.0,4405.0,1289.0,15729.0,20.0,5374.0,2801.0,40581.0,244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,fbad4165-ac84-44ed-8688-47f93fee5957,NaN,NaN,not reported,not reported,white,Not Reported,NaN,NaN,NaN,...,48.0,125.0,564.0,571.0,2093.0,77.0,1327.0,550.0,7538.0,174.0
1375,fc774c98-c5fa-4ba8-864d-45465e55250a,NaN,NaN,not hispanic or latino,female,white,Not Reported,NaN,13822.0,NaN,...,0.0,0.0,0.0,0.0,0.0,34.0,9.0,0.0,162.0,790.0
1376,fd7704f9-fd03-4b32-b0f4-98bd0444697b,NaN,NaN,not hispanic or latino,female,white,Not Reported,NaN,13014.0,NaN,...,4823.0,995.0,24148.0,2352.0,33958.0,1443.0,4487.0,4459.0,111836.0,102811.0
1377,fe25353b-d114-43f7-8d9b-eb6f2bd61d3d,NaN,NaN,not hispanic or latino,female,white,Not Reported,NaN,12859.0,NaN,...,1397.0,1193.0,11654.0,318.0,25576.0,589.0,2713.0,3141.0,100850.0,63567.0


One can check the columns and assess there is (almost) everything:

In [158]:
for col in df.columns:
    print(col)

case_id
demographic.age_at_index
demographic.days_to_birth
demographic.ethnicity
demographic.gender
demographic.race
demographic.vital_status
demographic.year_of_birth
diagnoses.0.age_at_diagnosis
diagnoses.0.ajcc_pathologic_m
diagnoses.0.ajcc_pathologic_n
diagnoses.0.ajcc_pathologic_stage
diagnoses.0.ajcc_pathologic_t
diagnoses.0.ajcc_staging_system_edition
diagnoses.0.classification_of_tumor
diagnoses.0.days_to_diagnosis
diagnoses.0.days_to_last_follow_up
diagnoses.0.last_known_disease_status
diagnoses.0.morphology
diagnoses.0.primary_diagnosis
diagnoses.0.prior_malignancy
diagnoses.0.prior_treatment
diagnoses.0.progression_or_recurrence
diagnoses.0.site_of_resection_or_biopsy
diagnoses.0.synchronous_malignancy
diagnoses.0.tissue_or_organ_of_origin
diagnoses.0.treatments.0.treatment_or_therapy
diagnoses.0.treatments.0.treatment_type
diagnoses.0.treatments.1.treatment_or_therapy
diagnoses.0.treatments.1.treatment_type
diagnoses.0.tumor_grade
diagnoses.0.tumor_stage
diagnoses.0.year_of

We may have forgotten some genes in the pivotings that were not involved in a particular dataset (CNV, SSM or ARN).

In [159]:
genes = list(df_gene["symbol"].sort_values().to_numpy())

CNV contains all the 50 genes:

In [160]:
genes_cnv = [col[11:] for col in df if col.startswith('cnv.change.')]
len(genes_cnv)

50

Missing genes in SSM:

In [161]:
genes_ssm = [col[15:] for col in df if col.startswith('ssm.importance.')]
ssm_missing = set(genes) - set(genes_ssm)
print(ssm_missing)

{'AC108925.1', 'FADD', 'FGF19', 'DNAJB6P5'}


Missing genes in ARN:

In [162]:
genes_arn = [col[17:] for col in df if col.startswith('arn.HTseq_counts.')]
arn_missing = set(genes) - set(genes_arn)
print(arn_missing)

{'AC108925.1', 'ORAOV1'}


Add the missing columns:

In [163]:
for gene in ssm_missing:
    col = 'ssm.importance.' + gene
    df[col] = 0.

In [164]:
for gene in arn_missing:
    col = 'arn.HTseq_counts.' + gene
    df[col] = 0.

In [165]:
df.to_csv("./data/gdc_final_database.csv")

In [166]:
df

,case_id,demographic.age_at_index,demographic.days_to_birth,demographic.ethnicity,demographic.gender,demographic.race,demographic.vital_status,demographic.year_of_birth,diagnoses.0.age_at_diagnosis,diagnoses.0.ajcc_pathologic_m,...,arn.HTseq_counts.TP53,arn.HTseq_counts.TPCN2,arn.HTseq_counts.TRPS1,arn.HTseq_counts.TTN,ssm.importance.AC108925.1,ssm.importance.FADD,ssm.importance.FGF19,ssm.importance.DNAJB6P5,arn.HTseq_counts.AC108925.1,arn.HTseq_counts.ORAOV1
0,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,50.0,-18345.0,not hispanic or latino,female,white,Alive,1955.0,18345.0,M0,...,2997.0,655.0,8320.0,341.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00a2d166-78c9-4687-a195-3d6315c27574,56.0,-20713.0,not hispanic or latino,female,white,Alive,1954.0,20713.0,M0,...,6779.0,1123.0,13940.0,306.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00b11ca8-8540-4a3d-b602-ec754b00230b,61.0,-22497.0,not hispanic or latino,female,white,Alive,1951.0,22497.0,MX,...,4543.0,3168.0,15170.0,1334.0,0.0,0.0,0.0,0.0,0.0,0.0
3,011b9b2d-ebe5-42bf-9662-d922faccc7a1,71.0,-26274.0,not hispanic or latino,female,white,Alive,1939.0,26274.0,MX,...,7822.0,1206.0,212884.0,19495.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01263518-5f7c-49dc-8d7e-84b0c03a6a63,76.0,-27942.0,not reported,female,not reported,Alive,1933.0,27942.0,M1,...,5374.0,2801.0,40581.0,244.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,fbad4165-ac84-44ed-8688-47f93fee5957,NaN,NaN,not reported,not reported,white,Not Reported,NaN,NaN,NaN,...,1327.0,550.0,7538.0,174.0,0.0,0.0,0.0,0.0,0.0,0.0
1375,fc774c98-c5fa-4ba8-864d-45465e55250a,NaN,NaN,not hispanic or latino,female,white,Not Reported,NaN,13822.0,NaN,...,9.0,0.0,162.0,790.0,0.0,0.0,0.0,0.0,0.0,0.0
1376,fd7704f9-fd03-4b32-b0f4-98bd0444697b,NaN,NaN,not hispanic or latino,female,white,Not Reported,NaN,13014.0,NaN,...,4487.0,4459.0,111836.0,102811.0,0.0,0.0,0.0,0.0,0.0,0.0
1377,fe25353b-d114-43f7-8d9b-eb6f2bd61d3d,NaN,NaN,not hispanic or latino,female,white,Not Reported,NaN,12859.0,NaN,...,2713.0,3141.0,100850.0,63567.0,0.0,0.0,0.0,0.0,0.0,0.0
